# Carbon Prediction: Estimating Bearing AI's Model
### Authors: William Olichney, Precious Ndunduri, Micah Thompson, Luke White
### Created for the William and Mary Master of Science in Business Analytics AI Class

### Bearing AI

Bearing AI has created an AI model that is advertised at being more effective than physics based models in predicting carbon emissions for martime shipping companies. As the European Union's Emission Trading System will require companies to account for 100% of their carbon emissions with a carbon credit, this Bearing AI model could be in high demand as maritime companies will need to accurately budget for this new environmental regulation.

### Our code
While our team does not have access to Bearing AI's model, this code is an attempt to approximate that model, in order to demonstrate how AI and statistical models make an important impact in aiding decision making.

#### Simulated Data
Using the Numpy library, we generated data of what ships **may** look like: Their speed, cargo weights, and fuel type. Then, we created formulas that would "create emissions" from those speed, weight, and fuel type metrics. In reality, someone would need to record how many tonnes of carbon a ship emmitted on voyage, the metrics of their ship, then this data could be used to feed into a statistical model.

This code is not to say this is what Bearing AI's model does, but it attempts to accomplish a similar goal: Predict carbon emissions for a potential voyage based off of historical data

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Simulate some fake data
np.random.seed(42)
n_samples = 1000

#Features: speed (knots), cargo_weight (tons), fuel_type (0: diesel, 1: LNG)
speed = np.random.uniform(10, 25, n_samples)
cargo_weight = np.random.uniform(5000, 20000, n_samples)
fuel_type = np.random.choice([0, 1], size=n_samples)

#Simulated emissions formula
#Diesel emits more than LNG. Emissions increase with weight and speed.
emissions = (
    0.2 * speed +
    0.0005 * cargo_weight +
    (fuel_type * 0.5 + 1.0) * 10 +
    np.random.normal(0, 2, n_samples)  # noise
)

#Create DataFrame
df = pd.DataFrame({
    'speed': speed,
    'cargo_weight': cargo_weight,
    'fuel_type': fuel_type,
    'emissions': emissions
})

#Split features and label
X = df[['speed', 'cargo_weight', 'fuel_type']].values
y = df['emissions'].values

#Normalize the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#Define a simple neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1)  # single output: predicted emissions
])

#Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

#Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

#Evaluate
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.2f} metric tons of CO₂")

#Predict for a new ship
new_ship = np.array([[18, 12000, 1]])  # 18 knots, 12,000 tons, LNG
new_ship_scaled = scaler.transform(new_ship)
predicted_emission = model.predict(new_ship_scaled)
print(f"Predicted emissions: {predicted_emission[0][0]:.2f} metric tons of CO₂")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 4.5718 - mae: 1.7217 
Test MAE: 1.68 metric tons of CO₂
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Predicted emissions: 23.59 metric tons of CO₂
